<a href="https://colab.research.google.com/github/ali-sdg/chinook_analytics_sqLite3/blob/V2/chinook_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import files
uploaded = files.upload()

Saving chinook.db to chinook (4).db


In [25]:
import sqlite3
import pandas as pd

# connect to database
conn = sqlite3.connect('chinook.db')


In [26]:
keywords_input = input("Please entar keywords separated by comma (ex: Rock, Beatles, Love): ")
keywords = [kw.strip() for kw in keywords_input.split(',') if kw.strip()]

if not keywords:
    print("No keywords entered. exiting..")
    conn.close()
    exit()


Please entar keywords separated by comma (ex: Rock, Beatles, Love): ac/ds,Rocket


In [27]:
keywords_input = input("Please entar keywords separated by comma (ex: Rock, Beatles, Love): ")
keywords = [kw.strip() for kw in keywords_input.split(',') if kw.strip()]

if not keywords:
    print("No keywords entered. exiting..")
    conn.close()
    exit()


Please entar keywords separated by comma (ex: Rock, Beatles, Love): AC/DC,Rocket


In [28]:
where_clauses = []
params = []
for kw in keywords:
    clause = "(t.Name LIKE ? OR ar.Name LIKE ? OR g.Name LIKE ?)"
    where_clauses.append(clause)
    like_param = f"%{kw}%"
    params.extend([like_param, like_param, like_param])

where_sql = " OR ".join(where_clauses)


In [29]:
sql_query = f"""
SELECT
  t.TrackId,
  t.Name AS TrackName,
  al.Title AS AlbumTitle,
  ar.Name AS ArtistName,
  g.Name AS GenreName,
  t.UnitPrice
FROM tracks t
JOIN albums al ON t.AlbumId = al.AlbumId
JOIN artists ar ON al.ArtistId = ar.ArtistId
JOIN genres g ON t.GenreId = g.GenreId
WHERE {where_sql}
"""

df = pd.read_sql_query(sql_query, conn, params=params)


In [30]:
def compute_score(row, keywords):
    score = 0
    for kw in keywords:
        if kw.lower() in str(row['TrackName']).lower():
            score += 1
        if kw.lower() in str(row['ArtistName']).lower():
            score += 2  # we give more weight if artist matched
        if kw.lower() in str(row['GenreName']).lower():
            score += 1
    return score

df['Score'] = df.apply(lambda r: compute_score(r, keywords), axis=1)

# filter only rows with some score
df = df[df['Score'] > 0]

# sort by score descending (highest first)
df = df.sort_values(by='Score', ascending=False)


In [31]:
if df.empty:
    print("No results found for your keywords.")
else:
    display(df.head(50))

conn.close()


,TrackId,TrackName,AlbumTitle,ArtistName,GenreName,UnitPrice,Score
0,1,For Those About To Rock (We Salute You),For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
1,6,Put The Finger On You,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
2,7,Let's Get It Up,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
3,8,Inject The Venom,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
4,9,Snowballed,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
5,10,Evil Walks,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
6,11,C.O.D.,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
7,12,Breaking The Rules,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
8,13,Night Of The Long Knives,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
9,14,Spellbound,For Those About To Rock We Salute You,AC/DC,Rock,0.99,2
